In [16]:
import re, os
import pandas as pd
import regex
import numpy as np

In [17]:
##파일 불러오기
file_name = "SKH_News Items_201601_20170120_175001.txt"
file_folder = "data"
data_path = os.path.join(file_folder, file_name)
df = pd.read_csv(data_path, sep="|")
print("before : ", len(df))
df??

before :  89767


In [18]:
##데이터가 없는 row 삭제
df = df.dropna()
print("after : ", len(df))

after :  21948


In [19]:
df.columns

Index(['Story Date Time', 'Headline', 'Story Body', 'Related RICs', 'Topics'], dtype='object')

In [49]:
df.index

df.shape
df.describe()
df.head()

# 컬럼의 데이터값별 중복갯수
df['Headline'].value_counts()   

TABLE-FOB U.S. cash millfeed prices                                                                                                                 15
U.S. cash crude deals                                                                                                                               12
BSE Block Deal PMC FINCORP-B 500,000 shares at 0.72 INR  <PRMF.BO>                                                                                  11
BRIEF-Toronto Stock Exchange unofficially closes down                                                                                               10
NSEI Block Deal VISESH INFOTEC 500,000 shares at 0.10 INR <VIFY.NS>                                                                                 10
SHH Margin Trading -- LY Junsheng <600699.SS> Margin Long Amount is 0.00   (x10000) yuan, Short Selling Volume is 0.00 (x10000) shares               9
SHH Margin Trading -- Synica <600490.SS> Margin Long Amount is 0.00   (x10000) yuan, Short Sel

In [22]:
##Index 생성
df["Story_num"] = df.index

In [24]:
df.columns
df.head()

,Story Date Time,Headline,Story Body,Related RICs,Topics,Story_num
1,12/31/2015 15:00:00,"TABLE-U.S. 30-year, 15-year mortgages",DEC 31 (REUTERS) - FREDDIE MAC <FMCC.OB> ...,FMCC.OB,FIN US COFS MTG INT DBT WASH AGN BSVC FINS MCE...,1
2,12/31/2015 15:00:03,US STOCKS-Indexes open lower on last day of year,* S&P 500 turns negative for year * Ap...,.DJI .INX .IXIC .SPX,STX US MKTREP REP MTGFX LEN AMERS RTRS,2
13,12/31/2015 15:01:27,UPDATE 2-Brazil's Itaú buys BTG Pactual's debt...,"(Adds quote, background) By Guillermo Par...",BBTG11.SA ITUB4.SA BPAC3.SA ITUB.N,LOA CDM FINS BANK INVS SFIN DFIN INVD DEAL1 EM...,13
27,12/31/2015 15:04:09,UPDATE 1-Saudi's market regulator refers suspe...,* CMA refers unnamed suspects to authoriti...,7020.SE,TCOM PHON WLES ACCI EMRG SA MEAST JUDIC FRAUD1...,27
38,12/31/2015 15:08:13,BUZZ-Prima BioMed Ltd: Roth Capital starts cov...,** Australian biotech's U.S.-listed shares...,PRR.AX,STX HOT HECA PHMR RCH US LEN BLR HEA CMPNY MRC...,38


In [48]:
df["Story Body"]

1             DEC 31 (REUTERS) - FREDDIE MAC <FMCC.OB> ...
2            * S&P 500 turns negative for year     * Ap...
13        (Adds quote, background)     By Guillermo Par...
27           * CMA refers unnamed suspects to authoriti...
38           ** Australian biotech's U.S.-listed shares...
40               * Decision to hike rates 25bps (to 3.2...
43           LAGOS, Dec 31 (Reuters) - The Nigerian nai...
46           ** Specialty rice packager's shares <ANFI....
51           ** Oil and gas company's shares <WPX.N> up...
53        Click the following link to watch video:     ...
56               * USD/CLP hovers near 710 up a touch i...
80           Dec 31 (Reuters) - Tesoro Corp <TSO.N>:   ...
82        (The following statement was released by the ...
90           Dec 31 (Reuters) - The U.S. Energy Informa...
97           ** Drug developer's shares <AVXL.O> down 9...
99           Dec 31 (Reuters) - Huishang Bank Corp Ltd ...
101       Click the following link to watch video:     .

In [29]:
def cleanText(text):
    
    ##광고 제거
    text = text.split("<^")[0]
    
    ##괄호 문장 제거 [] ()
    #text = remove_text_inside_brackets(text)
    text = regex.sub(r'\([^()]*+(?:(?R)[^()]*)*+\)', '', text)
    text = re.sub("[\<\(\[].*?[\)\]\>]", "", text)

    ##E-mail 제거
    email_regex = re.compile(r'(?i)\b[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,}\b')
    text= email_regex.sub('', text)

    ##HyperLink 제거
    text = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', text)

    # replace HTML symbols
    text = text.replace("&amp;", "and").replace("&gt;", ">").replace("&lt;", "<")

    ##양 끝 띄어쓰기 제거
    #text = text.strip()
    
    return text

def remove_one(text):
    remove_start_line = []
    remove_start_line.append("For the full text of this story please click the following link")
    remove_start_line.append("Keywords:")
    remove_start_line.append("Further company coverage:")
    remove_start_line.append("Source text for")
    
    for item in remove_start_line:
        if item in text:
            return True
    
    return False

def my_tokenizer(text):
    temp = text.split('  - ')
    temp = "  ".join(temp)
    temp = temp.split('  * ')
    temp = "  ".join(temp)
    temp = temp.split('  ')
    sentences = []
    for sentence in temp:
        item = sentence.strip()
        if not item is None and not item == "" and not remove_one(sentence):
            sentences.append(item)
            
    return sentences


In [30]:
sample_num = 20
sample = df["Story Body"].iloc[sample_num]
print(sample)
#print(my_tokenizer(cleanText(sample)))

 Click the following link to watch video:                               https://insider.thomsonreuters.com/link.html?cn=share&cid=1545178&shareToken=Mzo1NGEyMGU1Ni02N2MwLTRiNjUtODc5My0wNTcxNDgxNjlhMDg%3D&playerName=ReutersNews                                                                         Source:             Thomson Reuters                                                                                                           Description:        Public review of Britain's banking culture                             scrapped, Vodafone finishes year higher.            (To access all exclusive Reuters Insider programming visit: http://insider.thomsonreuters.com)   Short Link:  http://reut.rs/1OmPbC7


In [31]:
print(cleanText(sample))

 Click the following link to watch video:                                                                                                        Source:             Thomson Reuters                                                                                                           Description:        Public review of Britain's banking culture                             scrapped, Vodafone finishes year higher.               Short Link:  


In [32]:
print(df["Headline"].iloc[sample_num])

Reuters Insider - UK Day Ahead: Banks review, Vodafone ends on a high


In [33]:
from sacremoses import MosesTokenizer
from nltk.tokenize import sent_tokenize

In [34]:
moses_tokenize = MosesTokenizer(lang='en')

In [35]:
for item in my_tokenizer(cleanText(sample)):
    print(item)

Click the following link to watch video:
Source:
Thomson Reuters
Description:
Public review of Britain's banking culture
scrapped, Vodafone finishes year higher.
Short Link:


In [36]:
class myTokenizer:
    def __init__(self, parser):
        self.parser = parser
        
    def __call__(self, sample):
        # get the tokens using spaCy
        tokens = self.parser(sample)

        # lemmatize
        #remove_pos = ['-PRON-','SPACE','ADP','PUNCT','DET']

        lemmas = []
        for tok in tokens:
            #item = (tok.lemma_.lower().strip(), tok.pos_)
            #item = tok.lemma_
            item = str(tok.lemma_.lower().strip())+ "/" + str(tok.pos_)
            lemmas.append(item)

        tokens = lemmas

        return tokens

In [37]:
import spacy
nlp = spacy.load("en")
#parser = en_core_web_sm.load()
#basic_tokenizer = myTokenizer(parser)

In [38]:
temp = {}
for text in my_tokenizer(cleanText(sample)):
    tokens = nlp(text)
    for token in tokens:
        word = str(token)
        if word in temp:
            temp[word] = temp[word]+1
        else:
            temp[word] = 1

In [39]:
temp

{'Click': 1,
 'the': 1,
 'following': 1,
 'link': 1,
 'to': 1,
 'watch': 1,
 'video': 1,
 ':': 4,
 'Source': 1,
 'Thomson': 1,
 'Reuters': 1,
 'Description': 1,
 'Public': 1,
 'review': 1,
 'of': 1,
 'Britain': 1,
 "'s": 1,
 'banking': 1,
 'culture': 1,
 'scrapped': 1,
 ',': 1,
 'Vodafone': 1,
 'finishes': 1,
 'year': 1,
 'higher': 1,
 '.': 1,
 'Short': 1,
 'Link': 1}

In [40]:
kkk = {}
kkk["18"] = 1
if "18" in kkk:
    kkk["18"] = kkk["18"]+1
print(kkk["18"])

2


In [41]:
for item in my_tokenizer(cleanText(sample)):
    tokens = nlp(item)
    temp = []
    for token in tokens:
        temp.append(token.lemma_ + "/" + token.pos_)
    print(temp)
    print(tokens.ents)

['click/VERB', 'the/DET', 'follow/VERB', 'link/NOUN', 'to/PART', 'watch/VERB', 'video/NOUN', ':/PUNCT']
()
['source/NOUN', ':/PUNCT']
()
['Thomson/PROPN', 'reuter/NOUN']
(Thomson Reuters,)
['description/NOUN', ':/PUNCT']
()
['public/ADJ', 'review/NOUN', 'of/ADP', 'Britain/PROPN', "'s/PART", 'banking/NOUN', 'culture/NOUN']
(Britain,)
['scrap/VERB', ',/PUNCT', 'Vodafone/PROPN', 'finish/VERB', 'year/NOUN', 'higher/ADV', './PUNCT']
(Vodafone,)
['short/ADJ', 'Link/PROPN', ':/PUNCT']
()
